<a href="https://colab.research.google.com/github/RVT123123/audio_classification/blob/master/audio_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install  python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=8138511e871f4f834e7904bf305b1813a0444f640b882f0faa15302567e4064d
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [3]:
import pickle
import os
import numpy as np
from tqdm import tqdm
from scipy.io import wavfile
from python_speech_features import mfcc
from keras.models import load_model
import pandas as pd
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [24]:
classes

['Acoustic_guitar',
 'Bass_drum',
 'Cello',
 'Clarinet',
 'Double_bass',
 'Flute',
 'Hi-hat',
 'Saxophone',
 'Snare_drum',
 'Violin_or_fiddle']

In [0]:
def build_prediction(audio_dir):
    y_true = []
    y_pred = []
    fn_prob = {}
    
    
    
    print('Extacting features from audio')
    for fn in tqdm(os.listdir(audio_dir)):
        rate, wav = wavfile.read(os.path.join(audio_dir, fn))
        label=fn2class[fn]
        c = classes.index(label)
        y_prob = []

        for i in range(0, wav.shape[0]-config.step, config.step):
            sample = wav[i:i+config.step]
            x = mfcc(sample, rate,numcep = config.nfeat , nfilt=config.nfilt, nfft=config.nfft )
            x = (x - config.min)/ (config.max - config.min)
            
            
            if config.mode == 'conv':
                x = x.reshape(1, x.shape[0], x.shape[1], 1)
            elif config.mode == 'time':
                x = np.expand_dims(x,axis=0)
                
            y_hat = model.predict(x)
            y_prob.append(y_hat)
            y_pred.append(np.argmax(y_hat))
            y_true.append(c)
            
        fn_prob[fn] = np.mean(y_prob, axis=0).flatten()
        
    return y_true,y_pred,fn_prob

In [0]:
df = pd.read_csv('/content/gdrive/My Drive/audio_classification/Audio-Classification-master/instruments.csv')

In [18]:
df

,fname,label
0,5388d14d.wav,Saxophone
1,c685f05f.wav,Saxophone
2,36d20ab5.wav,Saxophone
3,d6665734.wav,Saxophone
4,7352e28f.wav,Saxophone
...,...,...
295,3c713bcf.wav,Clarinet
296,2fc00271.wav,Clarinet
297,b0c06255.wav,Clarinet
298,71c6451f.wav,Clarinet


In [0]:
classes = list(np.unique(df.label))
fn2class = dict(zip(df.fname, df.label))

In [21]:
len(fn2class)

300

In [0]:
p_path = os.path.join('/content/gdrive/My Drive/AUDIO_classifier/pickle/','conv.p')

In [10]:
p_path

'/content/gdrive/My Drive/AUDIO_classifier/pickle/conv.p'

In [0]:
with open(p_path,'rb') as handle:
    config = pickle.load(handle)

In [0]:
model = load_model(config.model_path)

In [28]:
y_true,y_pred, fn_prob = build_prediction('/content/gdrive/My Drive/audio_classification/Audio-Classification-master/clean/')
acc_score = accuracy_score(y_true=y_true, y_pred=y_pred)

  0%|          | 0/300 [00:00<?, ?it/s]

Extacting features from audio


100%|██████████| 300/300 [02:52<00:00,  1.74it/s]


In [29]:
acc_score

0.9490841604131365

In [35]:
df

,fname,label
0,5388d14d.wav,Saxophone
1,c685f05f.wav,Saxophone
2,36d20ab5.wav,Saxophone
3,d6665734.wav,Saxophone
4,7352e28f.wav,Saxophone
...,...,...
295,3c713bcf.wav,Clarinet
296,2fc00271.wav,Clarinet
297,b0c06255.wav,Clarinet
298,71c6451f.wav,Clarinet


['Acoustic_guitar',
 'Bass_drum',
 'Cello',
 'Clarinet',
 'Double_bass',
 'Flute',
 'Hi-hat',
 'Saxophone',
 'Snare_drum',
 'Violin_or_fiddle']

In [0]:
y_probs = []
for i,row in df.iterrows():
  y_prob = fn_prob[row.fname]
  y_probs.append(y_prob)
  for c , p in zip(classes, y_prob):
    df.at[i, c] = p


In [0]:
y_pred = [classes[np.argmax(y)] for y in y_probs]
df['y_pred'] = y_pred

In [53]:
len(y_pred)

300

In [54]:
df

,fname,label,Acoustic_guitar,Bass_drum,Cello,Clarinet,Double_bass,Flute,Hi-hat,Saxophone,Snare_drum,Violin_or_fiddle,y_pred
0,5388d14d.wav,Saxophone,1.049089e-04,3.488645e-02,0.005726,0.000468,3.873389e-04,0.027354,4.908820e-05,0.929035,5.422727e-05,0.001936,Saxophone
1,c685f05f.wav,Saxophone,2.855035e-03,9.503222e-05,0.007334,0.000235,1.070122e-05,0.002173,2.707841e-03,0.975437,5.825272e-04,0.008570,Saxophone
2,36d20ab5.wav,Saxophone,6.826568e-05,3.340378e-05,0.021175,0.000024,2.328529e-05,0.000034,3.032180e-06,0.978402,2.311828e-04,0.000006,Saxophone
3,d6665734.wav,Saxophone,6.469005e-03,7.456051e-05,0.057996,0.000198,3.589284e-03,0.019951,1.498134e-05,0.896105,5.681429e-04,0.015034,Saxophone
4,7352e28f.wav,Saxophone,1.037051e-09,1.536130e-11,0.000005,0.000023,5.726994e-08,0.009816,1.983828e-11,0.988325,2.064295e-13,0.001830,Saxophone
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,3c713bcf.wav,Clarinet,9.515157e-06,3.680542e-06,0.002267,0.997556,1.135282e-05,0.000005,1.058600e-04,0.000002,7.320091e-07,0.000039,Clarinet
296,2fc00271.wav,Clarinet,1.379349e-04,9.890555e-07,0.013752,0.984247,1.089483e-03,0.000229,6.306072e-05,0.000089,2.683035e-04,0.000123,Clarinet
297,b0c06255.wav,Clarinet,3.136675e-04,5.979131e-08,0.003218,0.968651,6.248268e-04,0.007721,1.471354e-05,0.013679,1.287038e-05,0.005765,Clarinet
298,71c6451f.wav,Clarinet,1.597403e-06,7.185930e-11,0.000015,0.907378,5.990226e-07,0.006096,2.373303e-07,0.000105,1.813244e-06,0.086402,Clarinet
